In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf

from Base import *

import sys
sys.path.insert(0, "/home/gregory/Desktop/CounterVision/Code")
from Checker import Checker
from Core import acc, prob
from Heuristics import *
from Learner import Learner
from Search import search
from Train import train
sys.path.insert(0, "/home/gregory/Desktop/EIE/SimpleData")
from SimpleData import vec2im


In [2]:
# Setup our datasets

p = 1.0

d = 6
n = 1000

rep = np.zeros((n, d))
X = np.zeros((n, 64, 64, 3))
y = np.zeros((n, 1))
for i in range(n):
    rep[i, :], y[i] = sample_1(p = p)
    X[i, :] = vec2im(rep[i, :])
    
X = np.float32(X) / 255
y = np.float32(y)
    
rep_train, rep_test, X_train, X_test, y_train, y_test = train_test_split(rep, X, y, test_size = 0.25)
rep_val, rep_test, X_val, X_test, y_val, y_test = train_test_split(rep_test, X_test, y_test, test_size = 0.5)


rep_neutral = np.zeros((100, d))
X_neutral = np.zeros((100, 64, 64, 3))
y_neutral = np.zeros((100, 1))
for i in range(100):
    rep_neutral[i, :], y_neutral[i] = sample_uniform()
    X_neutral[i, :] = vec2im(rep_neutral[i, :])
    
X_neutral = np.float32(X_neutral) / 255
y_neutral = np.float32(y_neutral)

In [3]:
# Train a model

model = tf.keras.Sequential([
          tf.keras.layers.InputLayer(input_shape=(64,64,3)),
          tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(1)
          ])

def loss(model, inputs, labels):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = model(inputs), labels = labels))

print(model.summary())

train(model, loss, X_train, y_train, X_val, y_val, "Image/model")
    
print("Test Acc: ", acc(model, X_test, y_test))
print("Neutral Acc: ", acc(model, X_neutral, y_neutral))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          3

In [4]:
# Load pretrained encoders and decoders for this dataset

encoder = tf.keras.Sequential([
          tf.keras.layers.InputLayer(input_shape=(64, 64, 3)),
          tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
          tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(6)
          ])

encoder.load_weights("/home/gregory/Desktop/EIE/SimpleData/1-Encoder/model")

decoder = tf.keras.Sequential(
        [
          tf.keras.layers.InputLayer(input_shape=(6)),
          tf.keras.layers.Dense(units=8*8*64, activation=tf.nn.relu),
          tf.keras.layers.Reshape(target_shape=(8, 8, 64)),
          tf.keras.layers.Conv2DTranspose(
              filters=64,
              kernel_size=3,
              strides=(2, 2),
              padding="SAME",
              activation='relu'),
          tf.keras.layers.Conv2DTranspose(
              filters=32,
              kernel_size=3,
              strides=(2, 2),
              padding="SAME",
              activation='relu'),
          tf.keras.layers.Conv2DTranspose(
              filters=32,
              kernel_size=3,
              strides=(2, 2),
              padding="SAME",
              activation='relu'),
          tf.keras.layers.Conv2DTranspose(
              filters=3, kernel_size=3, strides=(1, 1), padding="SAME", activation = "sigmoid"),
        ]
    )

decoder.load_weights("/home/gregory/Desktop/EIE/SimpleData/1-Decoder/model")

rep_test_hat = encoder(X_test).numpy()
X_test_hat = decoder(rep_test_hat).numpy()
print("Accuracy on reconstructed test set: ", acc(model, X_test_hat, y_test))


Accuracy on reconstructed test set:  1.0


In [5]:
y_hat_train = prob(model, X_train)
y_hat_val = prob(model, X_val)

checker = Checker("label")
learner = Learner(KNeighborsClassifier, encoder)
perturber_con = ContinuousPerturber(encoder = encoder, decoder = decoder)
perturber_cat = CategoricalPerturber(encoder = encoder, decoder = decoder)

# Search the continuous features
for index in range(6):
    
    print("\nSearching feature ", index)
    
    if index in [1, 2, 4]:
        heuristics = []
        for value in [-0.05, 0.05, -0.1, 0.1, -0.25, 0.25]:
            pert = np.zeros((1, d))
            pert[0, index] = value
            heuristics.append(pert)
        perturber = perturber_con
    
    if index in [0, 3, 5]:
        heuristics = []
        heuristics.append([index, 0])
        heuristics.append([index, 1])
        perturber = perturber_cat
        
    out = search(model, X_train, y_hat_train, heuristics, perturber, checker, learner, use_val = True, X_val = X_val, y_val = y_hat_val, use_acc = False, min_explainability = 0.8, verbose = True)



Searching feature  0

Heuristic:  [0, 0]
Success on Train:  0.0

Heuristic:  [0, 1]
Success on Train:  0.0

Searching feature  1

Heuristic:  [[ 0.   -0.05  0.    0.    0.    0.  ]]

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Success on Train:  0.0

Heuristic:  [[0.   0.05 0.   0.   0.   0.  ]]
Success on Train:  0.0

Heuristic:  [[ 0.  -0.1  0.   0.   0.   0. ]]
Success on Train:  0.0

Heuristic:  [[0.  0.1 0.  0.  0.  0. ]]
Success on Train:  0.0

Heuristic:  [[ 0.   -0.25  0.    0.    0.    0.  ]]
Success on Train:  0.0

Heuristic:  [[0.   0.

In [6]:
# Selective data augmentation

model.load_weights("Image/model")

def augment(model, perturber, h, X, y):
    X_pert, y_pert = perturber.apply(model, X, h)
    s_pert = checker.check(y, y_pert)
    indices = np.where(s_pert == 1)[0]
    return X_pert[indices, :], y[indices]

perturber = perturber_cat

X_train_0, y_train_0 = augment(model, perturber, [5, 0], X_train, y_train)
X_train_1, y_train_1 = augment(model, perturber, [5, 1], X_train, y_train)

X_val_0, y_val_0 = augment(model, perturber, [5, 0], X_val, y_val)
X_val_1, y_val_1 = augment(model, perturber, [5, 1], X_val, y_val)

X_train_aug = np.vstack((X_train, X_train_0, X_train_1))
y_train_aug = np.vstack((y_train, y_train_0, y_train_1))

print("New training size: ", X_train_aug.shape[0])

X_val_aug = np.vstack((X_val, X_val_0, X_val_1))
y_val_aug = np.vstack((y_val, y_val_0, y_val_1))


train(model, loss, X_train_aug, y_train_aug, X_val_aug, y_val_aug, "Image/model_aug", stopping_tol = 0.00001)

print("Test Acc: ", acc(model, X_test, y_test))
print("Neutral Acc: ", acc(model, X_neutral, y_neutral))


New training size:  1500
Test Acc:  1.0
Neutral Acc:  1.0


In [7]:
y_hat_train = prob(model, X_train)
y_hat_val = prob(model, X_val)

# Search the continuous features
for index in range(6):
    
    print("\nSearching feature ", index)
    
    if index in [1, 2, 4]:
        heuristics = []
        for value in [-0.05, 0.05, -0.1, 0.1, -0.25, 0.25]:
            pert = np.zeros((1, d))
            pert[0, index] = value
            heuristics.append(pert)
        perturber = perturber_con
    
    if index in [0, 3, 5]:
        heuristics = []
        heuristics.append([index, 0])
        heuristics.append([index, 1])
        perturber = perturber_cat
        
    out = search(model, X_train, y_hat_train, heuristics, perturber, checker, learner, use_val = True, X_val = X_val, y_val = y_hat_val, use_acc = False, min_explainability = 0.8, verbose = True)




Searching feature  0

Heuristic:  [0, 0]
Success on Train:  367.0
Counts:  383 383 367 367
Train Metrics:  [1. 1.]
Success on Val:  55.0
Counts:  70 70 55 55
Validation Metrics:  [1. 1.]
Accepted


Heuristic:  [0, 1]
Success on Train:  383.0
Counts:  367 367 383 383
Train Metrics:  [1. 1.]
Success on Val:  70.0
Counts:  55 55 70 70
Validation Metrics:  [1. 1.]
Accepted


Searching feature  1

Heuristic:  [[ 0.   -0.05  0.    0.    0.    0.  ]]
Success on Train:  1.0
Counts:  749 749 0 1
Train Metrics:  [1. 0.]
Rejected


Heuristic:  [[0.   0.05 0.   0.   0.   0.  ]]
Success on Train:  1.0
Counts:  749 749 0 1
Train Metrics:  [1. 0.]
Rejected


Heuristic:  [[ 0.  -0.1  0.   0.   0.   0. ]]
Success on Train:  2.0
Counts:  748 748 0 2
Train Metrics:  [1. 0.]
Rejected


Heuristic:  [[0.  0.1 0.  0.  0.  0. ]]
Success on Train:  2.0
Counts:  748 748 0 2
Train Metrics:  [1. 0.]
Rejected


Heuristic:  [[ 0.   -0.25  0.    0.    0.    0.  ]]
Success on Train:  49.0
Counts:  698 701 33 49
Trai